In [27]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

True

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.95,
    top_k=40,
    google_api_key=os.getenv("GEMINI_API_KEY") 
)

In [29]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    nooffours: int
    noofsixes: int
    sr: float
    bpb: float
    boundary_percentage: float
    summary: str

In [ ]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    state['sr'] = (state['runs'] / state['balls']) * 100
    return {'sr': state['sr']}

def calculate_bpb(state: BatsmanState) -> BatsmanState:
    state['bpb'] = state['balls']/(state['nooffours'] + state['noofsixes'])
    return {'bpb': state['bpb']}

def calculate_boundary_percentage(state: BatsmanState) -> BatsmanState:
    total_boudaries = state['nooffours']*4 + state['noofsixes']*6
    state['boundary_percentage'] = (total_boudaries / state['runs']) * 100 
    return {'boundary_percentage': state['boundary_percentage']}

def summary(state: BatsmanState) -> BatsmanState:
    summary = f"""
Batsman Summary:
- Runs: {state['runs']} 
- Balls: {state['balls']}
- Fours: {state['nooffours']}
- Sixes: {state['noofsixes']}
- Strike Rate: {state['sr']:.2f}
- Balls Per Boundary: {state['bpb']:.2f}
- Boundary Percentage: {state['boundary_percentage']:.2f}%
"""
    state['summary'] = summary
    
    return state


In [31]:
graph = StateGraph(BatsmanState)
graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_boundary_percentage)
graph.add_node('summary',summary)


# Add edges to define the workflow
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percentage')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percentage', 'summary')

graph.add_edge('summary',END)
workflow = graph.compile()

In [32]:
initial_state = BatsmanState(
    runs=100,
    balls=50,
    nooffours=10,
    noofsixes=5,
    sr=0.0,
    bpb=0.0,
    boundary_percentage=0.0,
    summary=""
)

workflow.invoke(initial_state)

{'runs': 100,
 'balls': 50,
 'nooffours': 10,
 'noofsixes': 5,
 'sr': 200.0,
 'bpb': 3.3333333333333335,
 'boundary_percentage': 70.0,
 'summary': '\nBatsman Summary:\n- Runs: 100 \n- Balls: 50\n- Fours: 10\n- Sixes: 5\n- Strike Rate: 200.00\n- Balls Per Boundary: 3.33\n- Boundary Percentage: 70.00%\n'}